In [26]:
import os
import re
import json
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:,.2f}'.format)

class UpbitMicrostructure:
    """업비트 거래 환경 정밀 모사"""
    FEE_RATE = 0.0005  # 0.05%
    MIN_ORDER_KRW = 5000
    
    @staticmethod
    def get_tick_size(price):
        if price >= 2_000_000: return 1000
        elif price >= 1_000_000: return 500
        elif price >= 500_000: return 100
        elif price >= 100_000: return 50
        elif price >= 10_000: return 10
        elif price >= 1_000: return 5
        elif price >= 100: return 1
        elif price >= 10: return 0.1
        else: return 0.01

    @staticmethod
    def adjust_to_tick(price):
        tick = UpbitMicrostructure.get_tick_size(price)
        return (price // tick) * tick

    @staticmethod
    def apply_slippage(price, is_buying=True, ticks=1):
        adjusted_price = price
        for _ in range(ticks):
            tick = UpbitMicrostructure.get_tick_size(adjusted_price)
            if is_buying: adjusted_price += tick
            else: 
                adjusted_price -= tick
                if adjusted_price < 0: adjusted_price = 0
        return adjusted_price

class QuantSimulator:
    def __init__(self, initial_capital=1_000_000):
        self.env = UpbitMicrostructure()
        self.initial_capital = initial_capital

    def calculate_metrics(self, equity_curve, dates, trades_log):
        if len(equity_curve) < 2: 
            return {'CAGR': 0, 'MDD': 0, 'Sharpe': 0, 'WinRate': 0, 'PF': 0}
            
        equity_s = pd.Series(equity_curve, index=dates)
        
        total_days = (dates[-1] - dates[0]).days
        if total_days < 1: total_days = 1
        total_ret = (equity_s.iloc[-1] / equity_s.iloc[0]) - 1
        cagr = ((1 + total_ret) ** (365 / total_days)) - 1
        
        roll_max = equity_s.cummax()
        drawdown = (equity_s - roll_max) / roll_max
        mdd = drawdown.min() * 100
        
        daily_eq = equity_s.resample('D').last().ffill()
        daily_ret = daily_eq.pct_change().dropna()
        rf_daily = 0.035 / 365
        
        sharpe = 0.0
        if len(daily_ret) > 1 and daily_ret.std() != 0:
            sharpe = (daily_ret.mean() - rf_daily) / daily_ret.std() * np.sqrt(365)

        wins = [t for t in trades_log if t > 0]
        losses = [t for t in trades_log if t <= 0]
        
        win_rate = (len(wins) / len(trades_log) * 100) if trades_log else 0
        gross_profit = sum(wins)
        gross_loss = abs(sum(losses))
        profit_factor = (gross_profit / gross_loss) if gross_loss > 0 else 99.99
        
        return {
            'CAGR': round(cagr * 100, 2),
            'MDD': round(mdd, 2),
            'Sharpe': round(sharpe, 2),
            'WinRate': round(win_rate, 2),
            'PF': round(profit_factor, 2)
        }

    def run_simulation_detail(self, df, df_hour, start_capital, model_threshold):
        equity = start_capital
        equity_curve = [equity]
        dates = [pd.to_datetime(df['date'].iloc[0])]
        trades_log = []
        
        df = df.sort_values('date').reset_index(drop=True)
        
        # 진입 가격 컬럼 자동 매핑 (real_entry_price 우선)
        entry_col = 'real_entry_price' if 'real_entry_price' in df.columns else 'entry_price'
        if entry_col not in df.columns: entry_col = 'close'

        start_p = self.env.apply_slippage(df[entry_col].iloc[0], True)
        end_p = self.env.apply_slippage(df[entry_col].iloc[-1], False)
        bh_ret = (end_p - start_p) / start_p * 100
        
        # === 자금 관리 설정 ===
        VOL_TARGET = 0.02
        KELLY_FRACTION = 0.5
        MIN_PROB = model_threshold 
        
        for date_val, daily_group in df.groupby('date'):
            candidates = daily_group[daily_group['prob'] >= MIN_PROB]
            
            if candidates.empty:
                equity_curve.append(equity)
                dates.append(date_val)
                continue
            
            best_signal = candidates.sort_values('prob', ascending=False).head(1)
            row = best_signal.iloc[0]
            
            entry_p = self.env.apply_slippage(self.env.adjust_to_tick(row[entry_col]), True)
            
            # [수정] ATR_84 사용 (FullMetricTester에서 merge된 이름 기준)
            # 만약 merge가 안 되었거나 값이 없으면 1% 대체
            atr_col = 'ATR_84' if 'ATR_84' in row else ('ETH_ATR_84' if 'ETH_ATR_84' in row else None)
            
            if atr_col and row[atr_col] > 0:
                atr_val = row[atr_col]
            else:
                atr_val = entry_p * 0.01
            
            # --- Position Sizing ---
            vol_qty = (equity * VOL_TARGET) / atr_val
            model_prob = row['prob']
            
            # TP/SL 자동 매핑
            tp_price = row['tp_price'] if 'tp_price' in row else (row['take_profit_price'] if 'take_profit_price' in row else entry_p + atr_val * 2.0)
            sl_price = row['sl_price'] if 'sl_price' in row else (row['stop_loss_price'] if 'stop_loss_price' in row else entry_p - atr_val * 1.0)
            
            potential_reward = tp_price - entry_p
            potential_risk = entry_p - sl_price
            
            if potential_risk <= 0: potential_risk = entry_p * 0.005
            risk_reward_ratio = potential_reward / potential_risk
            if risk_reward_ratio <= 0: risk_reward_ratio = 1.0
            
            kelly_f = model_prob - ((1 - model_prob) / risk_reward_ratio)
            
            if kelly_f <= 0:
                equity_curve.append(equity)
                dates.append(date_val)
                continue
                
            kelly_risk_amt = equity * kelly_f * KELLY_FRACTION
            kelly_qty = kelly_risk_amt / potential_risk
            qty = min(vol_qty, kelly_qty)
            max_qty_cash = (equity * 0.99) / entry_p
            qty = min(qty, max_qty_cash)
            
            pos_krw = qty * entry_p
            if pos_krw < self.env.MIN_ORDER_KRW: 
                equity_curve.append(equity)
                dates.append(date_val)
                continue
            
            fee_buy = pos_krw * self.env.FEE_RATE
            equity -= fee_buy
            
            entry_time = date_val + pd.Timedelta(hours=9)
            end_time = entry_time + pd.Timedelta(days=1)
            
            hourly_data = df_hour[(df_hour['datetime'] >= entry_time) & (df_hour['datetime'] < end_time)]
            
            exit_raw = hourly_data.iloc[-1]['close'] if not hourly_data.empty else row[entry_col]
            exit_p = self.env.apply_slippage(self.env.adjust_to_tick(exit_raw), False)
            
            triggered = False
            for _, h_row in hourly_data.iterrows():
                if h_row['low'] <= sl_price:
                    exit_p = self.env.apply_slippage(self.env.adjust_to_tick(sl_price), False)
                    triggered = True
                    break 
                if h_row['high'] >= tp_price:
                    exit_p = self.env.apply_slippage(self.env.adjust_to_tick(tp_price), False)
                    triggered = True
                    break
            
            sell_amt = qty * exit_p
            fee_sell = sell_amt * self.env.FEE_RATE
            
            net_pnl = sell_amt - fee_sell - pos_krw
            equity += net_pnl
            trades_log.append(net_pnl)
            
            equity_curve.append(equity)
            dates.append(date_val)
            
        metrics = self.calculate_metrics(equity_curve, dates, trades_log)
        metrics['End_Eq'] = equity
        metrics['BH_Ret'] = round(bh_ret, 2)
        metrics['My_Ret'] = round((equity - start_capital) / start_capital * 100, 2)
        metrics['Trades'] = len(trades_log)
        metrics['Start_Date'] = df['date'].min().strftime('%Y-%m-%d')
        metrics['End_Date'] = df['date'].max().strftime('%Y-%m-%d')
        metrics['Threshold_Used'] = MIN_PROB
        
        return metrics

class FullMetricTester:
    def __init__(self, root_dir, hourly_data_path):
        self.root_dir = root_dir
        self.sim = QuantSimulator(initial_capital=1_000_000) 
        self.data_map = {}
        self.candidates = [] 
        
        print(f"📂 Loading Hourly Data from {hourly_data_path}...")
        try:
            self.df_hour = pd.read_csv(hourly_data_path)
            self.df_hour['datetime'] = pd.to_datetime(self.df_hour['datetime'])
            self.df_hour = self.df_hour.sort_values('datetime').reset_index(drop=True)
            print("✅ Hourly Data Loaded Successfully.")
        except Exception as e:
            print(f"❌ Failed to load hourly data: {e}")
            self.df_hour = pd.DataFrame()

        # 🔹 4시간봉 데이터에서 ATR_84 로드
        try:
            # 만약 eth_4hour_all.csv가 상위 폴더에 있다면 "../eth_4hour_all.csv"
            # 현재 폴더에 있다면 "eth_4hour_all.csv"로 수정하세요
            df_4h = pd.read_csv("../eth_4hour_all.csv") 
            df_4h.rename(columns={'timestamp': 'date'}, inplace=True) # 혹시 컬럼명 다를 경우 대비
            df_4h['date'] = pd.to_datetime(df_4h['date'])
            
            # ETH_ATR_84 라고 되어있으면 이름 통일
            if 'ETH_ATR_84' in df_4h.columns:
                df_4h = df_4h.rename(columns={'ETH_ATR_84': 'ATR_84'})
                
            if 'ATR_84' in df_4h.columns:
                self.df_4h = df_4h[['date', 'ATR_84']]
                print("✅ 4H ATR Loaded Successfully.")
            else:
                 print("⚠️ ATR_84 column not found in eth_4hour_all.csv")
                 self.df_4h = pd.DataFrame()
                 
        except Exception as e:
            print(f"❌ Failed to load 4H ATR data: {e}")
            self.df_4h = pd.DataFrame()

    def collect(self):
        print(f"📂 Scanning all trials in: {self.root_dir}")
        trial_dirs = []
        for d in os.listdir(self.root_dir):
            full_path = os.path.join(self.root_dir, d)
            if os.path.isdir(full_path):
                if d.startswith('trial_') or (d.startswith('T') and d[1].isdigit()):
                    trial_dirs.append(full_path)

        def extract_trial_num(name):
            m1 = re.search(r'trial_(\d+)', name)
            m2 = re.search(r'^T(\d+)_', name)
            if m1: return int(m1.group(1))
            if m2: return int(m2.group(1))
            return 9999
            
        trial_dirs.sort(key=lambda x: extract_trial_num(os.path.basename(x)))
        
        count = 0
        loaded_meta = 0
        
        for trial_path in trial_dirs:
            trial_name = os.path.basename(trial_path)
            trial_id = extract_trial_num(trial_name)
            
            for root, dirs, files in os.walk(trial_path):
                pred_files = [f for f in files if f.startswith('predictions_') and f.endswith('.csv')]
                if not pred_files: continue
                
                fold_name = os.path.basename(root)
                
                meta_path = os.path.join(root, 'metadata.json')
                best_thresholds = {}
                
                if os.path.exists(meta_path):
                    try:
                        with open(meta_path, 'r') as f:
                            meta_data = json.load(f)
                            best_thresholds = meta_data.get('best_thresholds', {})
                            loaded_meta += 1
                    except:
                        pass
                
                for f in pred_files:
                    m_name = f.replace('predictions_', '').replace('.csv', '')
                    df = pd.read_csv(os.path.join(root, f))
                    df.rename(columns={'timestamp': 'date'}, inplace=True)
                    df['date'] = pd.to_datetime(df['date'])

                    # 🔹 여기서 4시간 ATR_84 붙이기 (Merge)
                    if not self.df_4h.empty:
                        df = pd.merge(
                            df,
                            self.df_4h,
                            on='date',
                            how='left'
                        )

                    th_val = best_thresholds.get(m_name, 0.5) + 0.1
                    
                    key = (trial_id, trial_name, m_name)
                    if key not in self.data_map: self.data_map[key] = []
                    if any(item['fold'] == fold_name for item in self.data_map[key]): continue
                    
                    self.data_map[key].append({'fold': fold_name, 'df': df, 'threshold': th_val})
                    count += 1
                    
        print(f"✅ Collected {count} prediction files. (Metadata found in {loaded_meta} folds)")

    def _simulate_single_model(self, key, folds):
        # (기존과 동일)
        t_id, t_name, m_name = key
        folds.sort(key=lambda x: x['df']['date'].min())
        
        wf_folds = [f for f in folds if 'final' not in f['fold']]
        final_folds = [f for f in folds if 'final' in f['fold']]
        
        wf_equity = 1_000_000
        wf_trades = 0
        wf_details = []
        
        for item in wf_folds:
            res = self.sim.run_simulation_detail(item['df'], self.df_hour, wf_equity, item['threshold'])
            wf_equity = res['End_Eq']
            wf_trades += res['Trades']
            wf_details.append(res)
            
        wf_ret = (wf_equity - 1_000_000) / 1_000_000 * 100
        
        final_equity = 1_000_000
        final_trades = 0
        final_details = []
        
        for item in final_folds:
            res = self.sim.run_simulation_detail(item['df'], self.df_hour, final_equity, item['threshold'])
            final_equity = res['End_Eq']
            final_trades += res['Trades']
            final_details.append(res)
            
        final_ret = (final_equity - 1_000_000) / 1_000_000 * 100
        
        wf_wins = sum([d['WinRate'] * d['Trades'] for d in wf_details]) / wf_trades if wf_trades > 0 else 0
        final_wins = sum([d['WinRate'] * d['Trades'] for d in final_details]) / final_trades if final_trades > 0 else 0

        return {
            'key': key,
            'WF_Return': wf_ret,
            'Final_Return': final_ret,
            'WF_Trades': wf_trades,
            'Final_Trades': final_trades,
            'WF_WinRate': wf_wins,
            'Final_WinRate': final_wins,
            'WF_Details': wf_details,
            'Final_Details': final_details
        }

    def run(self):
        if self.df_hour.empty:
            print("⚠️ Warning: Hourly data is empty.")
            
        print("\n🚀 Simulating all models... This may take a while.")
        
        for key, folds in self.data_map.items():
            result = self._simulate_single_model(key, folds)
            self.candidates.append(result)
            
        best_wf_model = max(self.candidates, key=lambda x: x['WF_Return'])
        best_final_model = max(self.candidates, key=lambda x: x['Final_Return'])
        
        self._print_champion_report("🏆 BEST WALK-FORWARD MODEL", best_wf_model)
        self._print_champion_report("🌟 BEST FINAL-HOLDOUT MODEL", best_final_model)

    def _print_champion_report(self, title, model_data):
        t_id, t_name, m_name = model_data['key']
        print(f"\n{'='*100}")
        print(f"{title}")
        print(f"Trial: {t_name} | Model: {m_name}")
        print(f"Walk-Forward Return: {model_data['WF_Return']:,.1f}% ({model_data['WF_Trades']} trades, Win: {model_data['WF_WinRate']:.1f}%)")
        print(f"Final-Holdout Return: {model_data['Final_Return']:,.1f}% ({model_data['Final_Trades']} trades, Win: {model_data['Final_WinRate']:.1f}%)")
        print(f"{'='*100}")
        
        print(f"\n[Detailed Performance by Fold]")
        print(f"{'Period':<23} {'Type':<10} {'Return':>8}% {'MDD':>7}% {'Sharpe':>6} {'PF':>5} {'Win%':>5} {'Cnt':>3}")
        print("-" * 100)
        
        for res in model_data['WF_Details']:
            print(f"{res['Start_Date']}~{res['End_Date']:<12} {'WF':<10} {res['My_Ret']:>8.1f} {res['MDD']:>7.1f} {res['Sharpe']:>6.2f} {res['PF']:>5.1f} {res['WinRate']:>5.1f} {res['Trades']:>3}")
            
        for res in model_data['Final_Details']:
            print(f"{res['Start_Date']}~{res['End_Date']:<12} {'FINAL':<10} {res['My_Ret']:>8.1f} {res['MDD']:>7.1f} {res['Sharpe']:>6.2f} {res['PF']:>5.1f} {res['WinRate']:>5.1f} {res['Trades']:>3}")
        print("\n")

if __name__ == "__main__":
    base_path = "2025-12-02_ATR_Based" 
    hourly_data_path = "../eth_hour.csv"
    
    if os.path.exists(base_path) and os.path.exists(hourly_data_path):
        tester = FullMetricTester(base_path, hourly_data_path)
        tester.collect()
        tester.run()
    else:
        print(f"Path check failed")


📂 Loading Hourly Data from ../eth_hour.csv...
✅ Hourly Data Loaded Successfully.
✅ 4H ATR Loaded Successfully.
📂 Scanning all trials in: 2025-12-02_ATR_Based
✅ Collected 3339 prediction files. (Metadata found in 1113 folds)

🚀 Simulating all models... This may take a while.

🏆 BEST WALK-FORWARD MODEL
Trial: T81_L12_P3.5_S1.3_N30 | Model: XGBoost
Walk-Forward Return: 155.1% (249 trades, Win: 54.2%)
Final-Holdout Return: -28.3% (24 trades, Win: 29.2%)

[Detailed Performance by Fold]
Period                  Type         Return%     MDD% Sharpe    PF  Win% Cnt
----------------------------------------------------------------------------------------------------
2021-11-25~2022-03-25   WF             77.5   -39.6   1.98   1.6  47.7 132
2022-03-28~2022-07-25   WF             23.3   -19.6   1.50   1.7  66.0  47
2022-07-28~2022-11-25   WF              0.0     0.0   0.00 100.0   0.0   0
2022-11-27~2023-03-27   WF              0.0     0.0   0.00 100.0   0.0   0
2023-03-29~2023-07-27   WF          

In [ ]:
import os
import re
import json
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:,.2f}'.format)

class UpbitMicrostructure:
    """업비트 거래 환경 정밀 모사"""
    FEE_RATE = 0.0005  # 0.05%
    MIN_ORDER_KRW = 5000
    
    @staticmethod
    def get_tick_size(price):
        if price >= 2_000_000: return 1000
        elif price >= 1_000_000: return 500
        elif price >= 500_000: return 100
        elif price >= 100_000: return 50
        elif price >= 10_000: return 10
        elif price >= 1_000: return 5
        elif price >= 100: return 1
        elif price >= 10: return 0.1
        else: return 0.01

    @staticmethod
    def adjust_to_tick(price):
        tick = UpbitMicrostructure.get_tick_size(price)
        return (price // tick) * tick

    @staticmethod
    def apply_slippage(price, is_buying=True, ticks=1):
        adjusted_price = price
        for _ in range(ticks):
            tick = UpbitMicrostructure.get_tick_size(adjusted_price)
            if is_buying: adjusted_price += tick
            else: 
                adjusted_price -= tick
                if adjusted_price < 0: adjusted_price = 0
        return adjusted_price

class QuantSimulator:
    def __init__(self, initial_capital=1_000_000):
        self.env = UpbitMicrostructure()
        self.initial_capital = initial_capital

    def calculate_metrics(self, equity_curve, dates, trades_log):
        if len(equity_curve) < 2: 
            return {'CAGR': 0, 'MDD': 0, 'Sharpe': 0, 'WinRate': 0, 'PF': 0}
            
        equity_s = pd.Series(equity_curve, index=dates)
        
        total_days = (dates[-1] - dates[0]).days
        if total_days < 1: total_days = 1
        total_ret = (equity_s.iloc[-1] / equity_s.iloc[0]) - 1
        cagr = ((1 + total_ret) ** (365 / total_days)) - 1
        
        roll_max = equity_s.cummax()
        drawdown = (equity_s - roll_max) / roll_max
        mdd = drawdown.min() * 100
        
        daily_eq = equity_s.resample('D').last().ffill()
        daily_ret = daily_eq.pct_change().dropna()
        rf_daily = 0.035 / 365
        
        sharpe = 0.0
        if len(daily_ret) > 1 and daily_ret.std() != 0:
            sharpe = (daily_ret.mean() - rf_daily) / daily_ret.std() * np.sqrt(365)

        wins = [t for t in trades_log if t > 0]
        losses = [t for t in trades_log if t <= 0]
        
        win_rate = (len(wins) / len(trades_log) * 100) if trades_log else 0
        gross_profit = sum(wins)
        gross_loss = abs(sum(losses))
        profit_factor = (gross_profit / gross_loss) if gross_loss > 0 else 99.99
        
        return {
            'CAGR': round(cagr * 100, 2),
            'MDD': round(mdd, 2),
            'Sharpe': round(sharpe, 2),
            'WinRate': round(win_rate, 2),
            'PF': round(profit_factor, 2)
        }

    def run_simulation_detail(self, df, df_hour, start_capital, model_threshold):
        equity = start_capital
        equity_curve = [equity]
        dates = [pd.to_datetime(df['date'].iloc[0])]
        trades_log = []
        
        df = df.sort_values('date').reset_index(drop=True)
        
        # 진입 가격 컬럼 자동 매핑 (real_entry_price 우선)
        entry_col = 'real_entry_price' if 'real_entry_price' in df.columns else 'entry_price'
        if entry_col not in df.columns: entry_col = 'close'

        start_p = self.env.apply_slippage(df[entry_col].iloc[0], True)
        end_p = self.env.apply_slippage(df[entry_col].iloc[-1], False)
        bh_ret = (end_p - start_p) / start_p * 100
        
        # === 자금 관리 설정 ===
        VOL_TARGET = 0.02
        KELLY_FRACTION = 0.5
        MIN_PROB = model_threshold 
        
        for date_val, daily_group in df.groupby('date'):
            candidates = daily_group[daily_group['prob'] >= MIN_PROB]
            
            if candidates.empty:
                equity_curve.append(equity)
                dates.append(date_val)
                continue
            
            best_signal = candidates.sort_values('prob', ascending=False).head(1)
            row = best_signal.iloc[0]
            
            entry_p = self.env.apply_slippage(self.env.adjust_to_tick(row[entry_col]), True)
            
            # [수정] ATR_84 사용 (FullMetricTester에서 merge된 이름 기준)
            # 만약 merge가 안 되었거나 값이 없으면 1% 대체
            atr_col = 'ATR_84' if 'ATR_84' in row else ('ETH_ATR_84' if 'ETH_ATR_84' in row else None)
            
            if atr_col and row[atr_col] > 0:
                atr_val = row[atr_col]
            else:
                atr_val = entry_p * 0.01
            
            # --- Position Sizing ---
            vol_qty = (equity * VOL_TARGET) / atr_val
            model_prob = row['prob']
            
            # TP/SL 자동 매핑
            tp_price = row['tp_price'] if 'tp_price' in row else (row['take_profit_price'] if 'take_profit_price' in row else entry_p + atr_val * 2.0)
            sl_price = row['sl_price'] if 'sl_price' in row else (row['stop_loss_price'] if 'stop_loss_price' in row else entry_p - atr_val * 1.0)
            
            potential_reward = tp_price - entry_p
            potential_risk = entry_p - sl_price
            
            if potential_risk <= 0: potential_risk = entry_p * 0.005
            risk_reward_ratio = potential_reward / potential_risk
            if risk_reward_ratio <= 0: risk_reward_ratio = 1.0
            
            kelly_f = model_prob - ((1 - model_prob) / risk_reward_ratio)
            
            if kelly_f <= 0:
                equity_curve.append(equity)
                dates.append(date_val)
                continue
                
            kelly_risk_amt = equity * kelly_f * KELLY_FRACTION
            kelly_qty = kelly_risk_amt / potential_risk
            qty = min(vol_qty, kelly_qty)
            max_qty_cash = (equity * 0.99) / entry_p
            qty = min(qty, max_qty_cash)
            
            pos_krw = qty * entry_p
            if pos_krw < self.env.MIN_ORDER_KRW: 
                equity_curve.append(equity)
                dates.append(date_val)
                continue
            
            fee_buy = pos_krw * self.env.FEE_RATE
            equity -= fee_buy
            
            entry_time = date_val + pd.Timedelta(hours=9)
            end_time = entry_time + pd.Timedelta(days=1)
            
            hourly_data = df_hour[(df_hour['datetime'] >= entry_time) & (df_hour['datetime'] < end_time)]
            
            exit_raw = hourly_data.iloc[-1]['close'] if not hourly_data.empty else row[entry_col]
            exit_p = self.env.apply_slippage(self.env.adjust_to_tick(exit_raw), False)
            
            triggered = False
            for _, h_row in hourly_data.iterrows():
                if h_row['low'] <= sl_price:
                    exit_p = self.env.apply_slippage(self.env.adjust_to_tick(sl_price), False)
                    triggered = True
                    break 
                if h_row['high'] >= tp_price:
                    exit_p = self.env.apply_slippage(self.env.adjust_to_tick(tp_price), False)
                    triggered = True
                    break
            
            sell_amt = qty * exit_p
            fee_sell = sell_amt * self.env.FEE_RATE
            
            net_pnl = sell_amt - fee_sell - pos_krw
            equity += net_pnl
            trades_log.append(net_pnl)
            
            equity_curve.append(equity)
            dates.append(date_val)
            
        metrics = self.calculate_metrics(equity_curve, dates, trades_log)
        metrics['End_Eq'] = equity
        metrics['BH_Ret'] = round(bh_ret, 2)
        metrics['My_Ret'] = round((equity - start_capital) / start_capital * 100, 2)
        metrics['Trades'] = len(trades_log)
        metrics['Start_Date'] = df['date'].min().strftime('%Y-%m-%d')
        metrics['End_Date'] = df['date'].max().strftime('%Y-%m-%d')
        metrics['Threshold_Used'] = MIN_PROB
        
        return metrics

class FullMetricTester:
    def __init__(self, root_dir, hourly_data_path):
        self.root_dir = root_dir
        self.sim = QuantSimulator(initial_capital=1_000_000) 
        self.data_map = {}
        self.candidates = [] 
        
        print(f"📂 Loading Hourly Data from {hourly_data_path}...")
        try:
            self.df_hour = pd.read_csv(hourly_data_path)
            self.df_hour['datetime'] = pd.to_datetime(self.df_hour['datetime'])
            self.df_hour = self.df_hour.sort_values('datetime').reset_index(drop=True)
            print("✅ Hourly Data Loaded Successfully.")
        except Exception as e:
            print(f"❌ Failed to load hourly data: {e}")
            self.df_hour = pd.DataFrame()

        # 🔹 4시간봉 데이터에서 ATR_84 로드
        try:
            # 만약 eth_4hour_all.csv가 상위 폴더에 있다면 "../eth_4hour_all.csv"
            # 현재 폴더에 있다면 "eth_4hour_all.csv"로 수정하세요
            df_4h = pd.read_csv("../eth_4hour_all.csv") 
            df_4h.rename(columns={'timestamp': 'date'}, inplace=True) # 혹시 컬럼명 다를 경우 대비
            df_4h['date'] = pd.to_datetime(df_4h['date'])
            
            # ETH_ATR_84 라고 되어있으면 이름 통일
            if 'ETH_ATR_84' in df_4h.columns:
                df_4h = df_4h.rename(columns={'ETH_ATR_84': 'ATR_84'})
                
            if 'ATR_84' in df_4h.columns:
                self.df_4h = df_4h[['date', 'ATR_84']]
                print("✅ 4H ATR Loaded Successfully.")
            else:
                 print("⚠️ ATR_84 column not found in eth_4hour_all.csv")
                 self.df_4h = pd.DataFrame()
                 
        except Exception as e:
            print(f"❌ Failed to load 4H ATR data: {e}")
            self.df_4h = pd.DataFrame()

    def collect(self):
        print(f"📂 Scanning all trials in: {self.root_dir}")
        trial_dirs = []
        for d in os.listdir(self.root_dir):
            full_path = os.path.join(self.root_dir, d)
            if os.path.isdir(full_path):
                if d.startswith('trial_') or (d.startswith('T') and d[1].isdigit()):
                    trial_dirs.append(full_path)

        def extract_trial_num(name):
            m1 = re.search(r'trial_(\d+)', name)
            m2 = re.search(r'^T(\d+)_', name)
            if m1: return int(m1.group(1))
            if m2: return int(m2.group(1))
            return 9999
            
        trial_dirs.sort(key=lambda x: extract_trial_num(os.path.basename(x)))
        
        count = 0
        loaded_meta = 0
        
        for trial_path in trial_dirs:
            trial_name = os.path.basename(trial_path)
            trial_id = extract_trial_num(trial_name)
            
            for root, dirs, files in os.walk(trial_path):
                pred_files = [f for f in files if f.startswith('predictions_') and f.endswith('.csv')]
                if not pred_files: continue
                
                fold_name = os.path.basename(root)
                
                meta_path = os.path.join(root, 'metadata.json')
                best_thresholds = {}
                
                if os.path.exists(meta_path):
                    try:
                        with open(meta_path, 'r') as f:
                            meta_data = json.load(f)
                            best_thresholds = meta_data.get('best_thresholds', {})
                            loaded_meta += 1
                    except:
                        pass
                
                for f in pred_files:
                    m_name = f.replace('predictions_', '').replace('.csv', '')
                    df = pd.read_csv(os.path.join(root, f))
                    df.rename(columns={'timestamp': 'date'}, inplace=True)
                    df['date'] = pd.to_datetime(df['date'])

                    # 🔹 여기서 4시간 ATR_84 붙이기 (Merge)
                    if not self.df_4h.empty:
                        df = pd.merge(
                            df,
                            self.df_4h,
                            on='date',
                            how='left'
                        )

                    th_val = best_thresholds.get(m_name, 0.5) 
                    
                    key = (trial_id, trial_name, m_name)
                    if key not in self.data_map: self.data_map[key] = []
                    if any(item['fold'] == fold_name for item in self.data_map[key]): continue
                    
                    self.data_map[key].append({'fold': fold_name, 'df': df, 'threshold': th_val})
                    count += 1
                    
        print(f"✅ Collected {count} prediction files. (Metadata found in {loaded_meta} folds)")

    def _simulate_single_model(self, key, folds):
        # (기존과 동일)
        t_id, t_name, m_name = key
        folds.sort(key=lambda x: x['df']['date'].min())
        
        wf_folds = [f for f in folds if 'final' not in f['fold']]
        final_folds = [f for f in folds if 'final' in f['fold']]
        
        wf_equity = 1_000_000
        wf_trades = 0
        wf_details = []
        
        for item in wf_folds:
            res = self.sim.run_simulation_detail(item['df'], self.df_hour, wf_equity, item['threshold'])
            wf_equity = res['End_Eq']
            wf_trades += res['Trades']
            wf_details.append(res)
            
        wf_ret = (wf_equity - 1_000_000) / 1_000_000 * 100
        
        final_equity = 1_000_000
        final_trades = 0
        final_details = []
        
        for item in final_folds:
            res = self.sim.run_simulation_detail(item['df'], self.df_hour, final_equity, item['threshold'])
            final_equity = res['End_Eq']
            final_trades += res['Trades']
            final_details.append(res)
            
        final_ret = (final_equity - 1_000_000) / 1_000_000 * 100
        
        wf_wins = sum([d['WinRate'] * d['Trades'] for d in wf_details]) / wf_trades if wf_trades > 0 else 0
        final_wins = sum([d['WinRate'] * d['Trades'] for d in final_details]) / final_trades if final_trades > 0 else 0

        return {
            'key': key,
            'WF_Return': wf_ret,
            'Final_Return': final_ret,
            'WF_Trades': wf_trades,
            'Final_Trades': final_trades,
            'WF_WinRate': wf_wins,
            'Final_WinRate': final_wins,
            'WF_Details': wf_details,
            'Final_Details': final_details
        }

    def run(self):
        if self.df_hour.empty:
            print("⚠️ Warning: Hourly data is empty.")
            
        print("\n🚀 Simulating all models... This may take a while.")
        
        for key, folds in self.data_map.items():
            result = self._simulate_single_model(key, folds)
            self.candidates.append(result)
            
        best_wf_model = max(self.candidates, key=lambda x: x['WF_Return'])
        best_final_model = max(self.candidates, key=lambda x: x['Final_Return'])
        
        self._print_champion_report("🏆 BEST WALK-FORWARD MODEL", best_wf_model)
        self._print_champion_report("🌟 BEST FINAL-HOLDOUT MODEL", best_final_model)

    def _print_champion_report(self, title, model_data):
        t_id, t_name, m_name = model_data['key']
        print(f"\n{'='*100}")
        print(f"{title}")
        print(f"Trial: {t_name} | Model: {m_name}")
        print(f"Walk-Forward Return: {model_data['WF_Return']:,.1f}% ({model_data['WF_Trades']} trades, Win: {model_data['WF_WinRate']:.1f}%)")
        print(f"Final-Holdout Return: {model_data['Final_Return']:,.1f}% ({model_data['Final_Trades']} trades, Win: {model_data['Final_WinRate']:.1f}%)")
        print(f"{'='*100}")
        
        print(f"\n[Detailed Performance by Fold]")
        print(f"{'Period':<23} {'Type':<10} {'Return':>8}% {'MDD':>7}% {'Sharpe':>6} {'PF':>5} {'Win%':>5} {'Cnt':>3}")
        print("-" * 100)
        
        for res in model_data['WF_Details']:
            print(f"{res['Start_Date']}~{res['End_Date']:<12} {'WF':<10} {res['My_Ret']:>8.1f} {res['MDD']:>7.1f} {res['Sharpe']:>6.2f} {res['PF']:>5.1f} {res['WinRate']:>5.1f} {res['Trades']:>3}")
            
        for res in model_data['Final_Details']:
            print(f"{res['Start_Date']}~{res['End_Date']:<12} {'FINAL':<10} {res['My_Ret']:>8.1f} {res['MDD']:>7.1f} {res['Sharpe']:>6.2f} {res['PF']:>5.1f} {res['WinRate']:>5.1f} {res['Trades']:>3}")
        print("\n")

if __name__ == "__main__":
    base_path = "2025-12-02_ATR_Based" 
    hourly_data_path = "../eth_hour.csv"
    
    if os.path.exists(base_path) and os.path.exists(hourly_data_path):
        tester = FullMetricTester(base_path, hourly_data_path)
        tester.collect()
        tester.run()
    else:
        print(f"Path check failed")


📂 Loading Hourly Data from ../eth_hour.csv...
✅ Hourly Data Loaded Successfully.
✅ 4H ATR Loaded Successfully.
📂 Scanning all trials in: 2025-12-02_ATR_Based
✅ Collected 3339 prediction files. (Metadata found in 1113 folds)

🚀 Simulating all models... This may take a while.
